Refer:
https://www.udemy.com/course/master-langchain-pinecone-openai-build-llm-applications/

## Project: Implementing a ChatGPT App with LangChain from Scratch



In [1]:
%pip install -qq langchain 
%pip install -qq langchain-community
%pip install -qq openai 


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

from langchain_openai import ChatOpenAI

from langchain.schema import SystemMessage
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate

In [5]:


# from langchain_core.output_parsers import StrOutputParser

# llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=1)

# prompt = ChatPromptTemplate(
#     input_variables=["content"],
#     messages=[
#         SystemMessage(content="You are a chatbot having a conversation with a human."),
#         SystemMessage(content='You respond only in English.'),
#         HumanMessagePromptTemplate.from_template("{content}")
#     ]
# )

# chain = prompt | llm | StrOutputParser()

# while True:
#     content = input('Your prompt: ')
#     if content.lower() in ['quit','q', 'exit', 'bye']:
#         print('Goodbye!')
#         break
    
#     response = chain.invoke({'content': content})
#     print(response)
#     print('-' * 50)
    

In [6]:
llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=1)

## Adding Chat Memory Using ConversationBufferMemory

In [15]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=False) 

from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage, AIMessage
from langchain.chains import LLMChain

# 1. Imports
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda


llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=1)

# 2. Create memory 
memory = ConversationBufferMemory(
    return_messages=True
)

# creating our conversation chain now
def get_memory_messages(query):
    return memory.load_memory_variables({})['history']


# 3. add  MessagesPlaceholder(variable_name='messages') to the prompt
prompt = ChatPromptTemplate(
    input_variables=["content"],
    messages=[
        SystemMessage(content="You are a chatbot having a conversation with a human."),
        MessagesPlaceholder(variable_name="chat_history"), # Where the memory will be stored.
        HumanMessagePromptTemplate.from_template("{content}")
    ]
)


conversation_chain = (
    RunnablePassthrough.assign(
        chat_history=RunnableLambda(get_memory_messages)
    ) # sends current query (input by user at runtime) and history messages to next step
      |
    prompt # creates prompt using the previous two variables
      |
    llm # generates response using the prompt from previous step
)

# # 4. Add the memory to the chain using LLMChain (memory cannot be used directly in LCEL pipes)
# chain = LLMChain(
#     llm=llm,
#     prompt=prompt,
#     memory=memory,
#     verbose=False
# )

# 5. Function to handle conversation with memory
def chat_with_memory(user_input):
    # Get response from chain
    response = conversation_chain.invoke({'content': user_input})
    
    # Save the conversation to memory
    memory.save_context(
        {"input": user_input},
        {"output": response.content}
    )
    
    return response

# Usage
response = chat_with_memory('Hello, how are you?')
print(response.content)

Hello! I'm just a chatbot, so I don't have feelings, but I'm here to help you. How can I assist you today?


In [16]:
print(memory.load_memory_variables({}))

{'history': [HumanMessage(content='Hello, how are you?', additional_kwargs={}, response_metadata={}), AIMessage(content="Hello! I'm just a chatbot, so I don't have feelings, but I'm here to help you. How can I assist you today?", additional_kwargs={}, response_metadata={})]}


In [18]:
print(memory.load_memory_variables({})['history'])

[HumanMessage(content='Hello, how are you?', additional_kwargs={}, response_metadata={}), AIMessage(content="Hello! I'm just a chatbot, so I don't have feelings, but I'm here to help you. How can I assist you today?", additional_kwargs={}, response_metadata={})]


In [ ]:
while True:
    content = input('Your prompt: ')
    if content.lower() in ['quit', 'exit', 'bye']:
        print('Goodbye!')
        break
    
    response = chain.invoke({'content': content})
    print(response['text'])
    print('-' * 50)

## Saving Chat Sessions

In [ ]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=False) 

from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage
from langchain.chains import LLMChain

# 1. Import FileChatMessageHistory
from langchain.memory import ConversationBufferMemory, FileChatMessageHistory

from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=1)

# 2. Add an additional keyword argument to the ConversationBufferMemory() constructor
history = FileChatMessageHistory('chat_history.json')
memory = ConversationBufferMemory(
    memory_key='chat_history',
    chat_memory=history,
    return_messages=True
)

prompt = ChatPromptTemplate(
    input_variables=["content", "chat_history"],
    messages=[
        SystemMessage(content="You are a chatbot having a conversation with a human."),
        MessagesPlaceholder(variable_name="chat_history"), 
        HumanMessagePromptTemplate.from_template("{content}")
    ]
)

chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory,
    verbose=False
)

while True:
    content = input('Your prompt: ')
    if content.lower() in ['quit', 'exit', 'bye']:
        print('Goodbye!')
        break
    
    response = chain.invoke({'content': content})
    print(response)
    print('-' * 50)
    

/var/folders/8v/xkrl1q210t5_4t4hvbx286800000gp/T/ipykernel_66614/211088997.py:32: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(


{'content': 'hi', 'chat_history': [], 'text': 'Hello! How are you today?'}
--------------------------------------------------
Goodbye!


In [20]:
# The messages property contains the list of messages in order.
print(history.messages)

[HumanMessage(content='hi', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello! How are you today?', additional_kwargs={}, response_metadata={})]
